In [1]:
import joblib

from app.component.models.model import TextSequences
from app.component.models.pipeline import Pipeline
from app.component.models.vectorizer import VectorizerBoW
from app.domain.models.tokenizer import TokenizerWord
from app.domain.models.topic_model import TopicModel
from app.infra.wikidb import WikiDb, WikiRecord

In [2]:
pipe_topic = Pipeline(
    steps=[
        (TokenizerWord(use_stoppoes=True), None),
        (VectorizerBoW(), None),
        (TopicModel(n_topic=10), None),
    ]
)

# pickup wiki data to analyze topic
wdb = WikiDb(mode="train")
records = wdb.select_by_document_id(document_ids=[
    # "Doc01GN7P2K02Y2FC8BXHMZ5V1A34",    # 南部煎餅
    "Doc01GN7P3C1ZPVGB8AXJC75M9ZQ9",    # 龍が如く OF THE END
    # "Doc01GN7P3C0YPAWH4KT16P7NNQBK",    # 自動運転車
    # "Doc01GN7P3BS2JNYS3HYV0RM1TT7T",    # デュケイン大学
    # "Doc01GN7P3BK88YCXKS64H6G2HK0Y",    # 深蒸し茶
    ])
X: TextSequences = [WikiRecord(*rec).paragraph.splitlines() for rec in records]
pipe_topic.fit(X)



In [3]:
# pipe_topic = joblib.load("data/pipe_topic.gz")
# pipe_topic


In [4]:
model_bow = pipe_topic.get_model(1)
model_topic = pipe_topic.get_model(-1)
model_bow, model_topic


(VectorizerBoW(), TopicModel(n_topic=10))

In [5]:
pipe_wikivec = joblib.load("data/pipe_wikivec.gz")
pipe_wikivec

Pipeline(steps=[(JpTokenizerMeCab(use_stoppoes=False), None), (VectorizerWord2vec(), None)], name=pipeline)

In [6]:
from app.component.models.vectorizer import VectorizerWord2vec
model_vectorizer: VectorizerWord2vec = pipe_wikivec.get_model(-1)
model_vectorizer

VectorizerWord2vec()

In [7]:
topics = []
for topic_probs in model_topic.get_topic_probabilities():
    indices = topic_probs.argsort()[::-1][:100]
    topic = [(model_bow.vocab[idx], topic_probs[idx]) for idx in indices]
    topics.append(topic)


print(topics[0])

[('する', 0.024466982), ('いる', 0.024466587), ('もの', 0.01772146), ('銃器', 0.017720934), ('アクション', 0.011007398), ('なる', 0.011007304), ('なっ', 0.0110071255), ('れ', 0.01100712), ('敵', 0.01100712), ('従来', 0.011007051), ('よう', 0.011006915), ('ある', 0.011006909), ('可能', 0.011006905), ('こと', 0.011006874), ('し', 0.011006637), ('取得', 0.011006569), ('タックル', 0.004434708), ('さ', 0.0044346745), ('操作', 0.004434647), ('チェイスバトル', 0.0044346363), ('無限', 0.0044346363), ('数', 0.0044346345), ('パートナー', 0.0044346256), ('各', 0.0044346256), ('サードパーソン・シューティング', 0.0044346238), ('攻撃', 0.0044346238), ('できる', 0.0044346107), ('落ち', 0.0044346107), ('視点', 0.0044346023), ('用意', 0.0044346005), ('ごと', 0.0044346005), ('多く', 0.0044345986), ('場合', 0.0044345963), ('弾薬', 0.0044345898), ('一', 0.0044345814), ('スキル', 0.0044345814), ('素手', 0.0044345814), ('使用', 0.0044345767), ('健在', 0.004434575), ('割り振ら', 0.004434573), ('れる', 0.004434571), ('ハンドガン', 0.004434566), ('弾き返し', 0.004434566), ('格闘', 0.0044345604), ('エイミング', 0.0044345534), ('弾

In [9]:
import numpy
import re

for idx, tpc in enumerate(topics):
    words = []
    probs = []
    for w, p in tpc:
        words.append(w)
        probs.append(p)
    _words = [w for w in words if w in model_vectorizer.model.wv]
    _probs = [p for w, p in zip(words, probs) if w in model_vectorizer.model.wv]
    vectors = model_vectorizer.model.wv[_words]
    probs = numpy.array(_probs).reshape(-1, 1)
    topic_vector = (vectors * probs).sum(axis=0)
    estimated_topics = model_vectorizer.model.wv.similar_by_vector(topic_vector, topn=10)

    topic = None
    probability = None
    for t, p in estimated_topics:
        if not re.search(r"^[あ-ん]+$", t):
            continue
        if not re.search(r"^[ア-ン]{,2}$", t):
            continue
        if len(t) <= 1:
            continue
        topic = t
        probability = p
        break
    print("-" * 100)
    print(f"topic[{idx}]: {topic}")
    print(" " * 4 + f" ... {[_t for _t, _p in tpc]}")


----------------------------------------------------------------------------------------------------
topic[0]: None
     ... ['する', 'いる', 'もの', '銃器', 'アクション', 'なる', 'なっ', 'れ', '敵', '従来', 'よう', 'ある', '可能', 'こと', 'し', '取得', 'タックル', 'さ', '操作', 'チェイスバトル', '無限', '数', 'パートナー', '各', 'サードパーソン・シューティング', '攻撃', 'できる', '落ち', '視点', '用意', 'ごと', '多く', '場合', '弾薬', '一', 'スキル', '素手', '使用', '健在', '割り振ら', 'れる', 'ハンドガン', '弾き返し', '格闘', 'エイミング', '弾', '専用', 'この', 'ケンカ', '掃討', '協力', '物', '蹴り技', '切り替わり', '時', '恒例', '狙い撃ち', '照準', '度', '特徴', '街中', 'ヒートアクション', '拾っ', '既存', '自動', '戦闘', '最大', '必要', '武器', '主人公', '爆発物', '変わり', '呼ば', 'モード', '本作', '基本', 'おり', 'いく', '補充', 'プレイ', '殲滅', 'ヒートスナイプ', 'その他', 'シリーズ', '作品', 'キャラクター', '旧', '隔離', 'エリア', '登場', 'ゾンビ', '兵器', '評価', '通常', 'ストーリー', 'ため', '配置', '撃破', '変わる', '異なる']
----------------------------------------------------------------------------------------------------
topic[1]: None
     ... ['ダンジョン', '主人公', '各', 'なっ', '生成', '入口', 'いる', '自動的', '最', '下層', 'たび', 'おり', '場所', '脱出'